In [27]:
import selenium as sel # automation tool
from selenium.webdriver.common.by import By
import time 
from selenium import webdriver # automation tool
from bs4 import BeautifulSoup # parsing tool
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd # data analysis tool
import numpy as np # numerical analysis tool
import sklearn as sk # machine learning tool
import nltk # natural language toolkit
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import requests
import re


In [30]:
restaurant_list = ['Green China Grill', 'Sizzling Pot King'] # list of restaurant names
df_comments = pd.DataFrame(columns=['restaurant', 'comment']) # create dataframe to store comments
driver = webdriver.Safari()

for resturant in restaurant_list:
    print(resturant)
    try:
        driver.get('https://www.yelp.com/')
        time.sleep(2)
        elem = driver.find_element(By.XPATH, "//input[contains(@id, 'search_description')]")
        elem.send_keys(resturant)
        elem = driver.find_element(By.XPATH, "//button[contains(@class, 'find-near-button')]")
        elem.click()
        time.sleep(2)
        original_window = driver.current_window_handle
        elem = driver.find_elements(By.XPATH, '//div[contains(@class, "businessName")]')[1]
        elem.click()
        time.sleep(2)
        for window_handle in driver.window_handles:
            if window_handle != original_window:
                driver.switch_to.window(window_handle)
                break
        while True: 
            url = driver.current_url
            req = requests.get(url)
            soup = BeautifulSoup(req.content, 'html.parser')
            comments = soup.find_all('p', class_ = re.compile('comment.*'))
            for comment in comments:
                df_comments = df_comments.append({'restaurant': resturant, 'comment': comment.text}, ignore_index=True)
            elem = driver.find_elements(By.XPATH, "//a[contains(@class, 'next-link navigation-button')]")[0]
            elem.click() 
    except: 
        continue # no more pages

Green China Grill
Sizzling Pot King


In [34]:
df_comments.to_csv('comments.csv', index=False)